# Sowing Success: How Machine Learning Helps Farmers Select the Best Crops

![Farmer in a field](farmer_in_a_field.jpg)

Measuring essential soil metrics such as nitrogen, phosphorous, potassium levels, and pH value is an important aspect of assessing soil condition. However, it can be an expensive and time-consuming process, which can cause farmers to prioritize which metrics to measure based on their budget constraints.

Farmers have various options when it comes to deciding which crop to plant each season. Their primary objective is to maximize the yield of their crops, taking into account different factors. One crucial factor that affects crop growth is the condition of the soil in the field, which can be assessed by measuring basic elements such as nitrogen and potassium levels. Each crop has an ideal soil condition that ensures optimal growth and maximum yield.

A farmer reached out to you as a machine learning expert for assistance in selecting the best crop for his field. They've provided you with a dataset called `soil_measures.csv`, which contains:

- `"N"`: Nitrogen content ratio in the soil
- `"P"`: Phosphorous content ratio in the soil
- `"K"`: Potassium content ratio in the soil
- `"pH"` value of the soil
- `"crop"`: categorical values that contain various crops (target variable).

Each row in this dataset represents various measures of the soil in a particular field. Based on these measurements, the crop specified in the `"crop"` column is the optimal choice for that field.  

In this project, you will build multi-class classification models to predict the type of `"crop"` and identify the single most importance feature for predictive performance.

In [1]:
# All required libraries are imported here for you.
import pandas as pd
import numpy as np
import operator

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import metrics

# Write your code here

In [2]:
# Prepare the environment
SEED = 42
np.random.seed(SEED)

# Read the data into a pandas DataFrame and perform exploratory data analysis

In [3]:
# Load the dataset
crops = pd.read_csv("soil_measures.csv")

In [4]:
# Verify the data shape
print(crops.shape)
crops.head()

(2200, 5)


,N,P,K,ph,crop
0,90,42,43,6.502985,rice
1,85,58,41,7.038096,rice
2,60,55,44,7.840207,rice
3,74,35,40,6.980401,rice
4,78,42,42,7.628473,rice


In [63]:
# Review the data types
crops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   N       2200 non-null   int64  
 1   P       2200 non-null   int64  
 2   K       2200 non-null   int64  
 3   ph      2200 non-null   float64
 4   crop    2200 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 86.1+ KB


In [64]:
# Review the main statistics
crops.describe().T

,count,mean,std,min,25%,50%,75%,max
N,2200.0,50.551818,36.917334,0.000000,21.000000,37.000000,84.250000,140.000000
P,2200.0,53.362727,32.985883,5.000000,28.000000,51.000000,68.000000,145.000000
K,2200.0,48.149091,50.647931,5.000000,20.000000,32.000000,49.000000,205.000000
ph,2200.0,6.469480,0.773938,3.504752,5.971693,6.425045,6.923643,9.935091


In [65]:
# Check for missing values
crops.isna().sum().sort_values()

N       0
P       0
K       0
ph      0
crop    0
dtype: int64

In [66]:
# Review the values in the target variable: crop
crop_types = crops.crop.unique()
print(len(crop_types))
print(crop_types)
crops.crop.value_counts().sort_values()

22
['rice' 'maize' 'chickpea' 'kidneybeans' 'pigeonpeas' 'mothbeans'
 'mungbean' 'blackgram' 'lentil' 'pomegranate' 'banana' 'mango' 'grapes'
 'watermelon' 'muskmelon' 'apple' 'orange' 'papaya' 'coconut' 'cotton'
 'jute' 'coffee']


rice           100
kidneybeans    100
pigeonpeas     100
mothbeans      100
mungbean       100
blackgram      100
lentil         100
pomegranate    100
banana         100
mango          100
grapes         100
watermelon     100
muskmelon      100
apple          100
orange         100
papaya         100
coconut        100
cotton         100
jute           100
maize          100
chickpea       100
coffee         100
Name: crop, dtype: int64

# Split the data

In [67]:
# Split in features and target variables
X = crops.drop('crop', axis=1)
y = crops.crop
print(X.shape, y.shape)

(2200, 4) (2200,)


In [68]:
# Split in training and testin set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, 
                                                    random_state=SEED,
                                                    stratify=y)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(1760, 4) (1760,)
(440, 4) (440,)


In [69]:
# Scaling the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

feature_names = scaler.get_feature_names_out()
X_train = pd.DataFrame(columns=feature_names, data=X_train)
X_test = pd.DataFrame(columns=feature_names, data=X_test)
feature_names

array(['N', 'P', 'K', 'ph'], dtype=object)

In [70]:
# Encode the target column
print(y_train.head().values)
labelenc = LabelEncoder()
y_train = labelenc.fit_transform(y_train) 
y_test = labelenc.transform(y_test)
y_train[:5]

['orange' 'grapes' 'kidneybeans' 'mothbeans' 'orange']


array([16,  7,  9, 13, 16])

In [71]:
print(np.unique(y_test))
label_names = labelenc.classes_
label_names

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]


array(['apple', 'banana', 'blackgram', 'chickpea', 'coconut', 'coffee',
       'cotton', 'grapes', 'jute', 'kidneybeans', 'lentil', 'maize',
       'mango', 'mothbeans', 'mungbean', 'muskmelon', 'orange', 'papaya',
       'pigeonpeas', 'pomegranate', 'rice', 'watermelon'], dtype=object)

# Evaluate Feature Performance

In [72]:
# Create a dictionary to store feature predictive performance
feature_performance = {}

# Loop through the features
for feature_name in X_train:
    # Instantiate the model
    model = LogisticRegression(random_state=SEED)
    
    # Train the model
    model.fit(X_train[[feature_name]], y_train)
    
    # Predict target values
    y_pred = model.predict(X_test[[feature_name]])
    
    # Evaluate performance
    score = metrics.f1_score(y_test, y_pred, average='weighted')
    feature_performance[feature_name] = score
    print(f'F1-score for {feature_name}:', score)

F1-score for N: 0.10483499038699813
F1-score for P: 0.13997712003244786
F1-score for K: 0.197728925789426
F1-score for ph: 0.07402057412825654


# Create the best_predictive_feature variable

In [73]:
best_predictive_feature = dict([max(
    feature_performance.items(), key=operator.itemgetter(1))
])
best_predictive_feature

{'K': 0.197728925789426}